# TD du 4 février 2025

# Exercice III

On considère le domaine $\Omega=[0,1]\times[0,1]$, la fonction $f\in L^{2}(\Omega)$, et l'EDP

$$(5.2)\quad\left\{
    \begin{array}{ll}
       - \Delta \displaystyle u=f &
       \textrm{dans}\ \Omega,\\
       u=0&\textrm{sur}\ \partial\Omega
    \end{array}\right.
$$

In [1]:
# On aura besoin des bibliothèques suivantes

import math
import numpy as np
import matplotlib.pyplot as plt
import sys
import time


# On aura également les besoins suivants 

from matplotlib import cm
from mpl_toolkits import mplot3d
from scipy import integrate